In [2]:
#Importing required libraries and packages.
import numpy as np
import pandas as pd
import tableprint as tp
from scipy.signal import detrend

### Defining the Mexican hat function based on:
###### http://en.wikipedia.org/wiki/Mexican_hat_wavelet
The A coefficient is added to control the amplitude of the Mexican hat.

In [0]:
def mexhat(t, A, D):

    return (2 * A /(np.sqrt(3 * D) * np.pi ** 0.25)) * (1 - t ** 2 / D ** 2) * np.exp(-t ** 2 / (2 * D ** 2))

In [12]:
# Defining a function to find the correlation between wind speed data and Mexican Hat shape
def detect_mexican_hat(A_vals, D_vals, wind_speed):

    # Create empty arrays to store results
    speed_index = []
    match_scores = []
    Ds = []
    MH_A = []
    amps = []

    # Loop over values of A and D
    for D in D_vals:

        # Apply the filter to the wind speed data in a rolling window
        for i in range(int(4*D) + start_time_index, end_time_index - start_time_index - int(4*D) + 1):
            wind_speed_window = wind_speed[i +start_time_index - int(4*D):i + start_time_index + int(4*D)]
            detrended_window = detrend(wind_speed_window)

            for A in A_vals:

                # Define a filter to identify Mexican hat events
                mexican_hat_filter = mexhat(np.arange(-4*D, 4*D+1), A, D)

                # Assuming filtered_window and mexican_hat_filter are two arrays with different shapes
                if detrended_window.shape[0] < mexican_hat_filter.shape[0]:
                    detrended_window_repeated = np.repeat(detrended_window[-1], mexican_hat_filter.shape[0])
                    detrended_window_repeated[:detrended_window.shape[0]] = detrended_window
                    mexican_hat_filter_repeated = mexican_hat_filter
                else:
                    detrended_window_repeated = detrended_window
                    mexican_hat_filter_repeated = np.repeat(mexican_hat_filter[-1], detrended_window.shape[0])
                    mexican_hat_filter_repeated[:mexican_hat_filter.shape[0]] = mexican_hat_filter

                window_match_score = np.corrcoef(detrended_window_repeated, mexican_hat_filter_repeated)[0,1]


                max_element = max(detrended_window, key=lambda x: abs(x))
                amp = max_element - detrended_window[0]

                speed_index.append(i)
                match_scores.append(window_match_score)
                Ds.append(D)
                MH_A.append(A)
                amps.append(amp)

    return speed_index, match_scores, Ds, MH_A, amps

In [11]:
# Read wind speed data from CSV file
df = pd.read_csv('Sample.csv')
df.head()

,timestamp,30m,40m,50m,60m,70m,80m,90m,100m
0,01-Oct-2015 00:00:01,3.88672,3.92144,3.79589,3.98800,4.05208,4.25160,4.35598,4.39459
1,01-Oct-2015 00:00:02,3.83608,3.92144,3.84648,3.88664,4.00125,4.25160,4.30509,4.34386
2,01-Oct-2015 00:00:03,3.83608,3.92144,3.79589,3.98800,3.95042,4.25160,4.30509,4.24240
3,01-Oct-2015 00:00:04,3.93736,3.76976,3.89707,3.93732,3.89959,4.25160,4.25420,4.19167
4,01-Oct-2015 00:00:05,3.88672,3.92144,3.89707,4.03868,3.89959,4.40391,4.30509,4.14094
...,...,...,...,...,...,...,...,...,...
12847,01-Oct-2015 03:34:08,1.81048,1.64624,1.77229,1.80876,1.76473,1.66233,1.55703,1.70590
12848,01-Oct-2015 03:34:09,1.86112,1.74736,1.72170,1.75808,1.81556,1.66233,1.50614,1.70590
12849,01-Oct-2015 03:34:10,1.91176,1.74736,1.72170,1.75808,1.76473,1.61156,1.55703,1.65517
12850,01-Oct-2015 03:34:11,1.96240,1.79792,1.72170,1.70740,1.76473,1.61156,1.55703,1.75663


In [13]:
start_time_index  = 0
end_time_index = 10000
wind_speed = df['50m']

# Plot the wind speed data and the detected Mexican hat events
time = np.arange(start_time_index,end_time_index)
time_stamp = df['timestamp']

# create arrays of values for A and D
A_vals = np.arange(1, 30.5, 0.5)
D_vals = np.arange(1, 5.1, 0.1)

# create meshgrid of A and D values
A, D = np.meshgrid(A_vals, D_vals)

# apply detect_mexican_hat function to entire arrays
speed_index, match_scores, Ds, MH_A, amps = detect_mexican_hat(A_vals, D_vals, wind_speed)

# flatten the result and convert to list
all_mexican_hats = np.column_stack((speed_index, match_scores, Ds, MH_A, amps)).tolist()

# sort the list by wind speed in descending order
all_mexican_hats.sort(key=lambda tup: tup[0], reverse=False)

#Checking the duplicates and only select the one with the highest score
optimum_results = []
for i in range(len(all_mexican_hats)):
    if i == 0 or (all_mexican_hats[i][0] - all_mexican_hats[i-1][0]) > 0:
        optimum_results.append(all_mexican_hats[i])
    else:
        if abs(all_mexican_hats[i][1]) > abs(optimum_results[-1][1]):
            optimum_results[-1] = all_mexican_hats[i]


optimum_peaks = [tup[0] for tup in optimum_results]
peak_time_stamp = [time_stamp[tup[0]] for tup in optimum_results]
speed_at_peak = [wind_speed[tup[0]] for tup in optimum_results]
optimum_score = [tup[1] for tup in optimum_results]
optimum_D = [tup[2] for tup in optimum_results]
optimum_A = [tup[3] for tup in optimum_results]
optimum_amps = [tup[4] for tup in optimum_results]


data = np.column_stack((optimum_peaks, peak_time_stamp, speed_at_peak, optimum_D, optimum_A, optimum_amps , optimum_score))
headers = ('Time Index', 'Time of the Mexican Hat', 'Wind Speed (m/s)', 'Optimum Width', 'Optimum A', 'MH Amplitude', 'Match Score')

In [14]:
table = tp.table(data,headers, width=25)

╭───────────────────────────┬───────────────────────────┬───────────────────────────┬───────────────────────────┬───────────────────────────┬───────────────────────────┬───────────────────────────╮
│                Time Index │   Time of the Mexican Hat │          Wind Speed (m/s) │             Optimum Width │                 Optimum A │              MH Amplitude │               Match Score │
├───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┤
│                       4.0 │      01-Oct-2015 00:00:05 │                   3.89707 │                       1.1 │                       4.5 │       0.05781714285714257 │      -0.40869786109516604 │
│                       5.0 │      01-Oct-2015 00:00:06 │                   3.89707 │        1.4000000000000004 │                       9.5 │       0.06806654545454416 │       -0.3104763499663737 │
│         